In [1]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel, DataCollatorForLanguageModeling, Trainer, TrainingArguments
import pandas as pd
df = pd.read_csv('C:/Users/nikhl/Downloads/LIC_Policies_Combined_Dataset.csv')

d:\ML\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Example of simple data cleaning
df = df.fillna('')


In [4]:

from sklearn.model_selection import train_test_split



# Split the data into training and validation sets
train_texts, val_texts = train_test_split(df['text'].tolist(), test_size=0.1)

# Tokenize the data
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=512)
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=512)

class LICDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __len__(self):
        return len(self.encodings['input_ids'])

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        return item

train_dataset = LICDataset(train_encodings)
val_dataset = LICDataset(val_encodings)

# Load model and training configurations
model = GPT2LMHeadModel.from_pretrained("gpt2")
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

training_args = TrainingArguments(
    output_dir="./results",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    save_steps=10_000,
    save_total_limit=2,
    evaluation_strategy="epoch",
    fp16=True,  # Use mixed precision if supported
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=data_collator,
)

# Train the model
trainer.train()

# Save the trained model
model.save_pretrained("D:/ML/trained_model")
tokenizer.save_pretrained("D:/ML/trained_model")



KeyError: 'text'